# DiploDatos 2021


### Categorización de publicaciones de productos realizadas en Mercado Libre

### 02 - Análisis y Curación

#### Condiciones generales que aplican a todos los prácticos:
   - Las notebooks tienen que ser 100% reproducibles, es decir al ejecutar las celdas tal cuál como se entrega la notebook se deben obtener los mismos resultados sin errores.
   - Código legible, haciendo buen uso de las celdas de la notebook y en lo posible seguir estándares de código para *Python* (https://www.python.org/dev/peps/pep-0008/).
   - Utilizar celdas tipo *Markdown* para ir guiando el análisis.
   - Limpiar el output de las celdas antes de entregar el notebook (ir a *Kernel* --> *Restart Kernel and Clear All Ouputs*).
   - Incluir conclusiones del análisis que se hizo en la sección "Conclusiones". Tratar de aportar valor en esta sección, ser creativo.

## 1. Consignas

#### Sección A:  Limpieza de texto / Preprocessing

Tener en cuenta lo siguiente: 

1. *Unidecode*

2. Pasar a minúsculas

3. Limpiar números

4. Limpiar símbolos **(** ' ! ¡ " @ % & * , . : ; < = > ? @ \ ^ _ { | } ~ \t \n [ ] ` $ **)**

5. Limpiar caracteres que suelen usarse como espacios **(** ' + ( ) - \ **)**

6. Reemplazar contracciones, por ejemplo, **c/u** por *cada uno*, **c/** por *con*, **p/** por *para*.

7. Etc.

#### Sección B: Tokenización & Secuencias

1. Utilizar métodos `fit_on_texts()`, `texts_to_sequences()`, y `pad_sequences()`:

- https://keras.io/api/preprocessing/text/#tokenizer-class

- https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

#### Sección C: Label Encoding

1. Utilizar método `LabelEncoder()` de *sklearn*:

- https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

#### Sección D: Word Embeddings

Generar los *word embeddings* correspondientes, de las siguientes dos formas:

1. *Custom Word Embeddings*
2. *Loading Pretrained Word Embeddings* (**opcional**)

En ambos puntos el objetivos final es llegar a crear la *embedding layer* de *keras*:

- https://keras.io/api/layers/core_layers/embedding/

## 2. Código y Análisis

Importaciones necesarias.

In [1]:
import pandas as pd
import numpy as np
import re
from unidecode import unidecode

Lectura de dataset reducido.

In [2]:
df_dataset = pd.read_csv('DataSet/dataset.csv')

Estudiamos el dataset brevemente antes de comenzar a operar sobre el mismo.

In [3]:
df_dataset.describe()

,title,label_quality,language,category
count,646760,646760,646760,646760
unique,646019,2,2,20
top,Teclado Casio Ctk-1300,unreliable,portuguese,PANTS
freq,2,551878,328992,35973


In [4]:
classes = np.sort(df_dataset.category.unique())

print(f'Dimensiones: {df_dataset.shape}')
print('----------')
print(f'Variables: {list(df_dataset.columns)}')
print('----------')
print(f'Categorías: {list(classes)}')

Dimensiones: (646760, 4)
----------
Variables: ['title', 'label_quality', 'language', 'category']
----------
Categorías: ['BABY_CAR_SEATS', 'BABY_STROLLERS', 'COFFEE_MAKERS', 'ELECTRIC_DRILLS', 'HAIR_CLIPPERS', 'KITCHEN_SINKS', 'MATTRESSES', 'MEMORY_CARDS', 'MOTORCYCLE_JACKETS', 'MUSICAL_KEYBOARDS', 'PANTS', 'PUREBRED_DOGS', 'RANGES', 'REFRIGERATORS', 'ROLLER_SKATES', 'SEWING_MACHINES', 'SHORTS', 'SUITCASES', 'WALL_CLOCKS', 'WINES']


## Sección A

Antes de aplicar la limpieza, demos un vistazo a algunas de las publicaciones de nuestro conjunto de datos.

In [5]:
df_dataset.sample(10, random_state=123)

,title,label_quality,language,category
181848,Fogão A Gás 4 Bocas Industrial Innal Alta Pre...,reliable,portuguese,RANGES
478587,Heladera Saccol Mod. Hsa32 320 Litros,unreliable,spanish,REFRIGERATORS
606516,Fogão Com Porta Full Glass E Timer E Relógio D...,unreliable,portuguese,RANGES
534521,The Beatles 09 - Relógio Disco De Vinil Decora...,unreliable,portuguese,WALL_CLOCKS
450844,Relógio De Parede Carrilhão Herweg Ref: 5352-084,reliable,portuguese,WALL_CLOCKS
352207,Aparadora Acabamento Maquina 100% Original Pan...,reliable,portuguese,HAIR_CLIPPERS
320466,Filhote Spitz Alemão Laranja Com Pedigree Cbkc,unreliable,portuguese,PUREBRED_DOGS
517042,Heladera Siam Hsi-rt60 Roja 420 Litros Retro C...,unreliable,spanish,REFRIGERATORS
519168,Teclado Korg Pa 600 Novíssimo Sem Detalhes,reliable,portuguese,MUSICAL_KEYBOARDS
407836,"Mala Berlim, Azul Marinho, M - Hg703m - Le Pos...",unreliable,portuguese,SUITCASES


**Unidecode**

A simple vista, se eliminan los tildes (en ambos idiomas).

Desde la [documentación](https://pypi.org/project/Unidecode/), se especifica:

It often happens that you have text data in *Unicode*, but you need to represent it in *ASCII*.

What **Unidecode** provides is a middle road: the function `unidecode()` takes *Unicode* data and tries to represent it in *ASCII* characters (i.e., the universally displayable characters between `0x00` and `0x7F`), where the compromises taken when mapping between two character sets are chosen to be near what a human with a *US* keyboard would choose.

In [6]:
df_dataset['clean_title'] = df_dataset.title.apply(unidecode)

**Minúsculas**

Se pasa todo a minúscula (en ambos idiomas).

In [7]:
df_dataset['clean_title'] = df_dataset.clean_title.apply(lambda x: x.lower())

**Limpiar Números**

Se borran todos los números.

In [8]:
df_dataset['clean_title'] = df_dataset.clean_title.apply(lambda x: re.sub(r'[0-9]+', '', x))

**Contracciones**

In [ ]:
## ACA DEBERIAMOS CAMBIAR LAS CONTRACCIONES NO ENCONTRE COMO TODAVIA ASI DESPUES SACAMOS TODOS LOS SIMBOLOS
df_dataset['clean_title']

**Limpiar Símbolos**

Se borran todos los símbolos que no sean letras.

In [9]:
df_dataset['clean_title'] = df_dataset.clean_title.apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))

**Limpieza Definitiva**

Damos un vistazo al resultado del procesamiento, luego de haber aplicado todos los pasos anteriores.

In [10]:
df_dataset.sample(10, random_state=123)

,title,label_quality,language,category,clean_title
181848,Fogão A Gás 4 Bocas Industrial Innal Alta Pre...,reliable,portuguese,RANGES,fogao a gas bocas industrial innal alta pres...
478587,Heladera Saccol Mod. Hsa32 320 Litros,unreliable,spanish,REFRIGERATORS,heladera saccol mod hsa litros
606516,Fogão Com Porta Full Glass E Timer E Relógio D...,unreliable,portuguese,RANGES,fogao com porta full glass e timer e relogio d...
534521,The Beatles 09 - Relógio Disco De Vinil Decora...,unreliable,portuguese,WALL_CLOCKS,the beatles relogio disco de vinil decoraca...
450844,Relógio De Parede Carrilhão Herweg Ref: 5352-084,reliable,portuguese,WALL_CLOCKS,relogio de parede carrilhao herweg ref
352207,Aparadora Acabamento Maquina 100% Original Pan...,reliable,portuguese,HAIR_CLIPPERS,aparadora acabamento maquina original panaso...
320466,Filhote Spitz Alemão Laranja Com Pedigree Cbkc,unreliable,portuguese,PUREBRED_DOGS,filhote spitz alemao laranja com pedigree cbkc
517042,Heladera Siam Hsi-rt60 Roja 420 Litros Retro C...,unreliable,spanish,REFRIGERATORS,heladera siam hsi rt roja litros retro combi
519168,Teclado Korg Pa 600 Novíssimo Sem Detalhes,reliable,portuguese,MUSICAL_KEYBOARDS,teclado korg pa novissimo sem detalhes
407836,"Mala Berlim, Azul Marinho, M - Hg703m - Le Pos...",unreliable,portuguese,SUITCASES,mala berlim azul marinho m hgm le postiche


## Sección B

Separamos nuestro conjunto de datos en los vectores `X`, e `y`.

- El primero, `X`, comprende los títulos procesados de las publicaciones.

- El segundo, `y`, representa las categorías de las publicaciones.

In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

X = df_dataset.clean_title.values
y = df_dataset.category.values

X[0], y[0]

('galoneira semi industrial', 'SEWING_MACHINES')

Utilizamos `Tokenizer()` para convertir los títulos de publicaciones en vectores.

In [12]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(X)

Necesitamos conocer el tamaño de nuestro vocabulario (se suma `+ 1` para contemplar las palabras *out of vocabulary*).

In [13]:
vocab_length = len(word_tokenizer.word_index) + 1

vocab_length

78808

Cada palabra se transforma al correspondiente índice en nuestro vocabulario.

In [14]:
embedded_sentences = word_tokenizer.texts_to_sequences(X)

embedded_sentences[0]

[609, 165, 48]

Aplicamos *padding* para que todos los vectores de palabras tengan tamaños equivalentes.

In [15]:
padded_sentences = pad_sequences(embedded_sentences, padding='post')

padded_sentences[0]

array([609, 165,  48,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [16]:
ammount_sentences, sentences_length = padded_sentences.shape

ammount_sentences, sentences_length

(646760, 22)

## Sección C

Necesitamos codificar las categorías de nuestras publicaciones.
Por lo tanto, utilizamos `LabelEncoder()` para transformar los nombres en valores numéricos.

In [17]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(classes)

LabelEncoder()

¿Cuántas categorías identificó el *encoder*?

In [18]:
le.classes_

array(['BABY_CAR_SEATS', 'BABY_STROLLERS', 'COFFEE_MAKERS',
       'ELECTRIC_DRILLS', 'HAIR_CLIPPERS', 'KITCHEN_SINKS', 'MATTRESSES',
       'MEMORY_CARDS', 'MOTORCYCLE_JACKETS', 'MUSICAL_KEYBOARDS', 'PANTS',
       'PUREBRED_DOGS', 'RANGES', 'REFRIGERATORS', 'ROLLER_SKATES',
       'SEWING_MACHINES', 'SHORTS', 'SUITCASES', 'WALL_CLOCKS', 'WINES'],
      dtype=object)

In [19]:
le.transform(classes)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

Aplicamos la transformación aprendida a todas las categorías de nuestro conjunto de datos.

In [20]:
encoded_labels = le.transform(y)

## Sección D

Finalmente, solo restan obtener los *word embeddings* para los títulos de nuestras publicaciones.
Utilizaremos `Embedding()` para calcularlos.

**Custom Word Embeddings**

De manera arbitraria, los vectores resultantes serán de **25** dimensiones.

In [21]:
from keras.layers.embeddings import Embedding

embedding_layer = Embedding(vocab_length, 25, input_length=sentences_length)

**Pretrained Word Embeddings**

Desde https://nlp.stanford.edu/projects/glove/, se descarga el *word embeding* entrenado **glove.6B.zip**.

De manera arbitraria, utilizaremos los vectores de **100** dimensiones.

In [22]:
embedding_dim = 100
glove_path = f'DataSet/glove.6B/glove.6B.{embedding_dim}d.txt'
glove_file = open(glove_path, encoding="utf8")

# Prepare embedding dictionary
embeddings_dictionary = dict()
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions

glove_file.close()

In [23]:
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((vocab_length, embedding_dim))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        embedding_matrix[index] = embedding_vector
        hits += 1
    else:
        misses += 1

print(f'Converted {hits} words ({misses} misses)')

Converted 28673 words (50134 misses)


In [24]:
trained_embedding_layer = Embedding(vocab_length,
                                    embedding_dim,
                                    weights=[embedding_matrix],
                                    input_length=sentences_length,
                                    trainable=False)

## Extra

Para practicar un poco con nuestra implementación, intentamos predecir con lo que tenemos hasta este punto.

In [25]:
from keras.models import Sequential
from keras.layers import Dense, Flatten

model = Sequential()

model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='softmax'))

In [26]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 22, 25)            1970200   
_________________________________________________________________
flatten (Flatten)            (None, 550)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 551       
Total params: 1,970,751
Trainable params: 1,970,751
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Ya que este proceso puede demorar, realizaremos un entrenamiento breve
model.fit(padded_sentences, encoded_labels, epochs=1, verbose=1)

20212/20212 [==============================] - 820s 40ms/step - loss: 0.0000e+00 - acc: 0.0487


In [28]:
# Evaluamos contra nuestro propio conjunto de entrenamiento
loss, accuracy = model.evaluate(padded_sentences, encoded_labels, verbose=0)

print(f'Accuracy: {accuracy * 100}')

Accuracy: 4.847702383995056


## 3. Conclusiones

In [ ]:
# TO DO

#### Material de ayuda para el desarrollo de este práctico:

1. Implementación en *keras* de *word embeddings*: https://stackabuse.com/python-for-nlp-word-embeddings-for-deep-learning-in-keras
2. Como utilizar *pre-trained word embeddings* en *keras*: https://keras.io/examples/nlp/pretrained_word_embeddings/
3. *Word Embeddings*: https://jalammar.github.io/illustrated-word2vec/
3. Curso de **procesamiento del lenguaje natural** con *keras*: https://www.coursera.org/learn/natural-language-processing-tensorflow/home/welcome